<a href="https://colab.research.google.com/github/appliedcode/mthree-c422/blob/c422_Mounika/DAY_7_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # A. Extract
# 1. Download or Create a Sample Data File
# Use any small public .csv (for demo, we’ll use a height/weight dataset):
url = "https://people.sc.fsu.edu/~jburkardt/data/csv/hw_200.csv"
!wget -qO customers.csv $url



# 2. Load Into pandas DataFrame
import pandas as pd

df_customers = pd.read_csv('customers.csv')
print(df_customers.head())
print(df_customers.columns)

   Index   Height(Inches)"   "Weight(Pounds)"
0      1             65.78             112.99
1      2             71.52             136.49
2      3             69.40             153.03
3      4             68.22             142.34
4      5             67.79             144.30
Index(['Index', ' Height(Inches)"', ' "Weight(Pounds)"'], dtype='object')


In [ ]:
# B. Transform
# 1. Clean Data
# Remove duplicates

# Rename messy columns for cleaner analysis

# Remove duplicates based on 'Index' column
df_customers.drop_duplicates('Index', inplace=True)

# Clean up column names
df_customers.columns = [col.strip().replace('"', '').replace("'", "") for col in df_customers.columns]

# View cleaned DataFrame
print(df_customers.head())


   Index  Height(Inches)  Weight(Pounds)
0      1           65.78          112.99
1      2           71.52          136.49
2      3           69.40          153.03
3      4           68.22          142.34
4      5           67.79          144.30


In [ ]:
# 2. Add Enriched & Derived Columns
# Compute a "BMI" (Body Mass Index) column

# Add a BMI column, Height in inches, Weight in pounds
df_customers['BMI'] = (df_customers['Weight(Pounds)'] / (df_customers['Height(Inches)'] * 0.0254) ** 2)
print(df_customers.head())


   Index  Height(Inches)  Weight(Pounds)        BMI
0      1           65.78          112.99  40.474812
1      2           71.52          136.49  41.359796
2      3           69.40          153.03  49.248182
3      4           68.22          142.34  47.406304
4      5           67.79          144.30  48.670704


In [ ]:
# C. Load
# 1. Store the Result in a Local SQLite Database
import sqlite3

conn = sqlite3.connect('customers_etl_lab.db')
df_customers.to_sql('customers_etl', conn, if_exists='replace', index=False)

200

In [ ]:
# 2. Run SQL Queries on the Transformed Data
# Example: Get customers with BMI > 25 (overweight threshold)
query = "SELECT * FROM customers_etl WHERE BMI > 25;"
df_overweight = pd.read_sql(query, conn)
print(df_overweight.head())

   Index  Height(Inches)  Weight(Pounds)        BMI
0      1           65.78          112.99  40.474812
1      2           71.52          136.49  41.359796
2      3           69.40          153.03  49.248182
3      4           68.22          142.34  47.406304
4      5           67.79          144.30  48.670704


In [ ]:
url = "https://people.sc.fsu.edu/~jburkardt/data/csv/hw_200.csv"
!wget -qO customers.csv $url


In [ ]:
import pandas as pd
df_customers = pd.read_csv('customers.csv')


In [ ]:
df_customers.drop_duplicates('Index', inplace=True)


In [ ]:
import sqlite3
conn = sqlite3.connect('customers_etl_lab.db')
df_customers.to_sql('customers_etl', conn, if_exists='replace', index=False)


200

 Global COVID-19 Time Series Data

In [ ]:
# Step 0: Install SQLite (if not installed)
!pip install -q pandas sqlalchemy

# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import sqlite3

# Step 2: Extract – Load the CSV
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
df = pd.read_csv(url)

# Step 3: Transform – Clean & Reshape

# Melt the data to long format
df_long = df.melt(id_vars=["Province/State", "Country/Region", "Lat", "Long"],
                  var_name="Date", value_name="Confirmed")

# Convert 'Date' to datetime
df_long["Date"] = pd.to_datetime(df_long["Date"])

# Group by Country and Month
df_long["YearMonth"] = df_long["Date"].dt.to_period("M")
monthly_df = df_long.groupby(["Country/Region", "YearMonth"])["Confirmed"].sum().reset_index()

# Calculate Monthly Increase and Growth Rate
monthly_df["Confirmed_lag"] = monthly_df.groupby("Country/Region")["Confirmed"].shift(1)
monthly_df["Monthly_Increase"] = monthly_df["Confirmed"] - monthly_df["Confirmed_lag"]
monthly_df["Growth_Rate"] = (monthly_df["Monthly_Increase"] / monthly_df["Confirmed_lag"]) * 100
monthly_df = monthly_df.fillna(0)

# Convert YearMonth to string for SQLite
monthly_df["YearMonth"] = monthly_df["YearMonth"].astype(str)

# Step 4: Load – Store in SQLite DB
conn = sqlite3.connect("covid_monthly.db")
monthly_df.to_sql("monthly_cases", conn, if_exists="replace", index=False)

# Verify – Display first few rows
print("Transformed Data Sample:")
display(monthly_df.head())

print("✅ ETL Pipeline Completed and Data Loaded into SQLite Database")


/tmp/ipython-input-1579226253.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_long["Date"] = pd.to_datetime(df_long["Date"])


Transformed Data Sample:


,Country/Region,YearMonth,Confirmed,Confirmed_lag,Monthly_Increase,Growth_Rate
0,Afghanistan,2020-01,0,0.0,0.0,0.000000
1,Afghanistan,2020-02,30,0.0,30.0,inf
2,Afghanistan,2020-03,1141,30.0,1111.0,3703.333333
3,Afghanistan,2020-04,25152,1141.0,24011.0,2104.382121
4,Afghanistan,2020-05,222720,25152.0,197568.0,785.496183


✅ ETL Pipeline Completed and Data Loaded into SQLite Database


In [ ]:
import pandas as pd
import sqlite3

# Reconnect to your SQLite DB
conn = sqlite3.connect("covid_monthly.db")

# Define your SQL query as a Python string
query = """
SELECT [Country/Region], YearMonth, Confirmed, Monthly_Increase, Growth_Rate
FROM monthly_cases
WHERE [Country/Region] = 'India'
ORDER BY YearMonth;
"""

# Use pandas to execute the SQL query
result_df = pd.read_sql_query(query, conn)

# Display the result
result_df.head()


,Country/Region,YearMonth,Confirmed,Monthly_Increase,Growth_Rate
0,India,2020-01,2,0.0,0.000000
1,India,2020-02,84,82.0,4100.000000
2,India,2020-03,10252,10168.0,12104.761905
3,India,2020-04,447607,437355.0,4266.045650
4,India,2020-05,3088494,2640887.0,590.001273


In [ ]:
import sqlite3
import pandas as pd

# Connect to the database (make sure you've already created and populated it)
conn = sqlite3.connect("covid_monthly.db")

# SQL query as a string
query = "SELECT * FROM monthly_cases"

# Use pandas to run the query and load results into a DataFrame
df = pd.read_sql_query(query, conn)

# Show the first few rows
df.head()


,Country/Region,YearMonth,Confirmed,Confirmed_lag,Monthly_Increase,Growth_Rate
0,Afghanistan,2020-01,0,0.0,0.0,0.000000
1,Afghanistan,2020-02,30,0.0,30.0,inf
2,Afghanistan,2020-03,1141,30.0,1111.0,3703.333333
3,Afghanistan,2020-04,25152,1141.0,24011.0,2104.382121
4,Afghanistan,2020-05,222720,25152.0,197568.0,785.496183
